# PREPROCESSING MESH DATA TO LINKED DATA
In this notebook, we evaluate the mesh assets in a session repository.
For every obj or ply polygonal mesh, a MESHNODE metadata class is created 
that governs all the metadata of the mesh (paths, pose, etc.).
As ouput, the method generates RDF GRAPHS (.ttl) and O3D.GEOMETRY.MESH files

>This codebase operates on the scan2bim2.yml environment (python 3.8)

In [1]:
#IMPORT PACKAGES
from rdflib import Graph, plugin
import os.path
import numpy as np
import open3d as o3d
import uuid    
import importlib

#IMPORT MODULES
from context import geomapi 
from geomapi.nodes import *
import geomapi.utils as ut
from geomapi.utils import geometryutils as gmu
import geomapi.tools as tl

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
%load_ext autoreload

In [3]:
%autoreload 2

## 1. INITIALIZE SESSION

In [20]:
## INPUTS
projectPath= os.path.join(os.path.abspath(os.path.join(os.getcwd(), os.pardir)),"tests" )#"D:\\Data\\2018-06 Werfopvolging Academiestraat Gent" 
sessionPath = os.path.join(projectPath,"Samples1") #"K:\Projects\2025-03 Project FWO SB Jelle\7.Data\21-11 House Maarten\RAW data\session_22-03-13 canon
graphPath = os.path.join(sessionPath,"meshGraph.ttl")
print(graphPath)
sensor="Hololens 2" # (string) P30, BLK, Hololens2, CANON (zie exif), etc.
accuracy=0.05

d:\Scan-to-BIM repository\geomapi\tests\Samples1\meshGraph.ttl


## 2. READ PRESENT OBJ/PLY FILES CONTAINING POLYGONAL MESHES
A MeshNode is created per polygonal mesh in the session

In [5]:
# path=os.path.join(sessionPath,"week22.obj")
# print(path)
# graphPath=os.path.join(sessionPath,"meshGraph.ttl")
# mesh=  o3d.io.read_triangle_mesh(path)
# node1=MeshNode(path=path, graphPath=graphPath, sensor=sensor, accuracy=accuracy)
# # node2=MeshNode(graphPath=graphPath, sessionPath=sessionPath, sensor=sensor, accuracy=accuracy)
# # node3=MeshNode(mesh=mesh, sessionPath=sessionPath, sensor=sensor, accuracy=accuracy)


In [18]:
allSessionFilePaths=ut.get_list_of_files(sessionPath) 
nodelist=[]
paths=[]
meshes=[]
for path in allSessionFilePaths:   
    if path.endswith(".obj") or path.endswith(".ply"): 
        paths.append(path)
        meshes.append(o3d.io.read_triangle_mesh(path)) #loading only take 7s of 600 meshes

In [23]:
allSessionFilePaths=ut.get_list_of_files(sessionPath) 
nodelist=[]

for idx,path in enumerate(paths):   
    nodelist.append(MeshNode(path=paths[idx], mesh=meshes[idx], graphPath=graphPath, getMetaData=True,sensor=sensor, accuracy=accuracy))              
print(str(len(nodelist))+' MeshNodes are created.')


603 MeshNodes are created.
Note that these nodes do not necesserily contain the actual data but only the metadata


## 4. CREATE RDF GRAPHS FOR EACH MESHNODES AND EXPORT THEM TO .TTL
An RDF graph and .ttl file is created for all meshes in the session
(data itself is not stored in the graph, only metadata)

In [26]:
graph=tl.nodes_to_graph(nodelist,graphPath=graphPath,save=True)
print(len(graph))
print(graph.serialize())
print (str(len(nodelist))+' MeshNodes succesfully serialized in: '+graphPath)

6030
@prefix e57: <http://libe57.org#> .
@prefix openlabel: <https://www.asam.net/index.php?eID=dumpFile&t=f&f=3876&token=413e8c85031ae64cc35cf42d0768627514868b2f#> .
@prefix v4d: <https://w3id.org/v4d/core#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<file:///Basic_Wall_000_WA_DummyWall_20mm_1130411> a v4d:MeshNode ;
    e57:cartesianBounds """[-14.01650047 -13.62110043  69.10179901  69.13130188  13.97000027
  17.20999908]""" ;
    e57:cartesianTransform """[[  1.           0.           0.         -13.81880021]
 [  0.           1.           0.          69.11655045]
 [  0.           0.           1.          15.58999968]
 [  0.           0.           0.           1.        ]]""" ;
    e57:pointCount 8 ;
    v4d:accuracy "0.05"^^xsd:float ;
    v4d:faceCount 12 ;
    v4d:name "Basic Wall_000_WA_DummyWall 20mm_1130411" ;
    v4d:path "BIM\\Basic Wall_000_WA_DummyWall 20mm_1130411.obj" ;
    openlabel:sensor "Hololens 2" ;
    openlabel:timestamp "2022-04-06 15:16:28" .

<file:/